In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.document_loaders import JSONLoader

# Load data
loader = JSONLoader(
    file_path="../data/experience.json",
    jq_schema=".",
    text_content=False
)

# Load documents (this will be raw data from the JSON)
docs = loader.load()

In [3]:
import json
from langchain_core.documents import Document

documents = [
    Document(
        page_content=json.dumps(entry, ensure_ascii=False),
        metadata={
            "title": entry.get("title", ""),
            "start_date": entry.get("start_date", ""),
            "end_date": entry.get("end_date", ""),
            "association": entry.get("association", "")
        }
    )
    for entry in json.loads(docs[0].page_content)
]

In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3:8b")

In [5]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3:8b")

In [6]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_documents(documents=documents)

In [17]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an expert resume writer. Given the context and a tailoring instruction (question), generate a customized resume section or full resume.

Context (raw resume data or background info):
{context}

Tailoring Request (e.g. job role, industry, emphasis):
{question}

Generate a professional, cleanly formatted resume tailored to the request.
""",
)

In [18]:
from typing_extensions import List, TypedDict

# Define state for application
class State(TypedDict):
  question: str
  context: List[Document]
  answer: str

In [19]:
# Define application steps
def retrieve(state: State):
  retrieved_docs = vector_store.similarity_search(state["question"])
  return {"context": retrieved_docs}

def generate(state: State):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = prompt.invoke({"question": state["question"], "context": docs_content})
  response = llm.invoke(messages)
  return {"answer": response.content}

In [20]:
from langgraph.graph import START, StateGraph

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [30]:
job_description = """
We are looking for a Software Engineer with experience in full-stack web development.
The ideal candidate will be proficient in Python, React, and cloud infrastructure (AWS or GCP).
Responsibilities include developing scalable web applications, working closely with product teams,
and contributing to system design and architecture. Experience with CI/CD pipelines and Docker is a plus.
"""

In [31]:
response = graph.invoke({"question": f"Tailor this resume for the following job: {job_description}"})
print(response["answer"])

Here's a customized resume based on the provided context and tailoring instruction:

**[Your Name]**
**Software Engineer**

**Summary:**
Highly motivated and detail-oriented software engineer with experience in full-stack web development, proficiency in Python, React, and cloud infrastructure (AWS/GCP). Proven track record of developing scalable web applications, collaborating closely with product teams, and contributing to system design and architecture.

**Technical Skills:**

* Programming languages: Python
* Front-end frameworks: React
* Cloud infrastructure: AWS/GCP
* Full-stack development experience
* CI/CD pipelines: [insert relevant experience or training]
* Docker: [insert relevant experience or training]

**Experience:**

**Software Engineer Internship, Technical Consulting & Research, Inc. (Jan 2025 - Apr 2025)**
* Collaborated with stakeholders to design and develop scalable backend systems using Auth0
* Created SQL database schemas using SQLAlchemy and implemented authent